Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# MONAI 101 Tutorial

Welcome to MONAI 101! This tutorial introduces beginners to the basics of building an end-to-end medical image classification pipeline with MONAI.

## What You'll Learn

In this tutorial, you'll discover how simple it can be to create a complete medical image classification system. We'll cover each step with just a few lines of code:

- **Dataset Download**: Automatically retrieve and set up the MedNIST dataset
- **Data Preprocessing**: Transform medical images for training
- **Model Definition**: Set up a DenseNet-121 neural network for classification
- **Training**: Train your model with medical imaging data
- **Evaluation**: Test your trained model's performance

## Requirements

- **GPU Memory**: Approximately 7GB
- **Runtime**: About 10 minutes
- **Level**: Beginner (no prior MONAI experience required)

## Quick Start Options

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/2d_classification/monai_101.ipynb)

*Click the badge above to run this tutorial in Google Colab without any local setup.*

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[ignite, tqdm]"

## Setup imports

In [ ]:
import logging
import numpy as np
import os
from pathlib import Path
import sys
import tempfile
import torch

from monai.apps import MedNISTDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.engines import SupervisedTrainer
from monai.handlers import StatsHandler
from monai.inferers import SimpleInferer
from monai.networks import eval_mode
from monai.networks.nets import densenet121
from monai.transforms import LoadImageD, EnsureChannelFirstD, ScaleIntensityD, Compose

print_config()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MONAI version: 1.2.0rc4+21.g7f067564
Numpy version: 1.22.2
Pytorch version: 2.0.0a0+1767026
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 7f06756472fd5514c3c2f2a6710e3fa4d1748e90
MONAI __file__: /workspace/monai/monai-in-dev/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.3.0
Nibabel version: 5.1.0
scikit-image version: 0.20.0
Pillow version: 9.2.0
Tensorboard version: 2.9.0
gdown version: 4.7.1
TorchVision version: 0.15.0a0
tqdm version: 4.65.0
lmdb version: 1.4.1
psutil version: 5.9.4
pandas version: 1.5.2
einops version: 0.6.1
transformers version: 4.21.3
mlflow version: 2.3.0
pynrrd version: 1.0.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup Data Directory

You can specify a directory for storing datasets and results using the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to:
- Save results permanently
- Reuse downloaded datasets across different sessions
- Avoid re-downloading large datasets

If not specified, a temporary directory will be used (data will be lost after the session ends).

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/data


## Use MONAI Transforms to Preprocess Data

Medical images require specialized methods for input/output (I/O), preprocessing, and augmentation. Unlike natural images, medical images often:
- Follow specific formats (DICOM, NIfTI, etc.)
- Are handled with specific protocols
- Have high-dimensional data arrays
- Require domain-specific preprocessing

In this example, we'll create a preprocessing pipeline using three MONAI transforms:

1. **`LoadImageD`**: Loads medical images from various formats
2. **`EnsureChannelFirstD`**: Ensures the image has the correct channel dimension
3. **`ScaleIntensityD`**: Normalizes pixel intensities to a standard range

These transforms are combined into a pipeline that will be applied to our data.

In [ ]:
transform = Compose(
    [
        LoadImageD(keys="image", image_only=True),
        EnsureChannelFirstD(keys="image"),
        ScaleIntensityD(keys="image"),
    ]
)

## Prepare Dataset Using MONAI Apps

We'll use the `MedNISTDataset` from MONAI Apps to automatically download and set up our dataset. This convenience class will:
- Download the dataset to your specified directory
- Apply the preprocessing transforms we defined above
- Split the data into training, validation, and test sets

### About the MedNIST Dataset

The MedNIST dataset is a collection of medical images from multiple sources:
- [TCIA](https://wiki.cancerimagingarchive.net/display/Public/Data+Usage+Policies+and+Restrictions) (The Cancer Imaging Archive)
- [RSNA Bone Age Challenge](http://rsnachallenges.cloudapp.net/competitions/4)
- [NIH Chest X-ray Dataset](https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest)

### Dataset Information
- **Size**: 58,954 images
- **Classes**: 6 medical image types (AbdomenCT, BreastMRI, CXR, ChestCT, Hand, HeadCT)
- **Format**: 2D grayscale images
- **License**: Creative Commons [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

The dataset is kindly made available by [Dr. Bradley J. Erickson M.D., Ph.D.](https://www.mayo.edu/research/labs/radiology-informatics/overview) (Department of Radiology, Mayo Clinic).

*If you use the MedNIST dataset in your research, please acknowledge the source.*

In [ ]:
dataset = MedNISTDataset(root_dir=root_dir, transform=transform, section="training", download=True)

2023-04-21 15:37:46,567 - INFO - Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
2023-04-21 15:37:46,567 - INFO - File exists: /workspace/data/MedNIST.tar.gz, skipped downloading.
2023-04-21 15:37:46,568 - INFO - Non-empty folder exists in /workspace/data/MedNIST, skipped extracting.


Loading dataset: 100%|██████████| 47164/47164 [00:18<00:00, 2525.58it/s]


## Define Network and Supervised Trainer

Now we'll set up our machine learning model and training configuration.

### Model Selection: DenseNet-121

We'll use DenseNet-121, a proven convolutional neural network architecture that:
- Has shown excellent performance on ImageNet and medical imaging tasks
- Features dense connections between layers for better gradient flow
- Is computationally efficient for medical image classification

### Training Configuration

MONAI provides `SupervisedTrainer` to simplify the training process. This high-level API handles:
- Training loops and optimization
- Loss computation and backpropagation  
- Metric tracking and logging
- Device management (CPU/GPU)

In [ ]:
max_epochs = 5
model = densenet121(spatial_dims=2, in_channels=1, out_channels=6).to("cuda:0")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
trainer = SupervisedTrainer(
    device=torch.device("cuda:0"),
    max_epochs=max_epochs,
    train_data_loader=DataLoader(dataset, batch_size=512, shuffle=True, num_workers=4),
    network=model,
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-5),
    loss_function=torch.nn.CrossEntropyLoss(),
    inferer=SimpleInferer(),
    train_handlers=StatsHandler(),
)

## Run the Training

Now let's start the training process! The trainer will:
- Load batches of medical images
- Forward them through the DenseNet-121 model
- Calculate the loss and update model weights
- Track training progress

This should take about 10 minutes on a GPU.

In [ ]:
trainer.run()

## Evaluate Model Performance on Test Dataset

Let's see how well our trained model performs! We'll:
- Load the test dataset (images the model has never seen)
- Run predictions on these images
- Compare predictions with ground truth labels
- Display the results to see classification accuracy

This evaluation helps us understand if our model can generalize to new medical images.

In [ ]:
dataset_dir = Path(root_dir, "MedNIST")
class_names = sorted(f"{x.name}" for x in dataset_dir.iterdir() if x.is_dir())
testdata = MedNISTDataset(root_dir=root_dir, transform=transform, section="test", download=False, runtime_cache=True)

max_items_to_print = 10
with eval_mode(model):
    for item in DataLoader(testdata, batch_size=1, num_workers=0):
        prob = np.array(model(item["image"].to("cuda:0")).detach().to("cpu"))[0]
        pred = class_names[prob.argmax()]
        gt = item["class_name"][0]
        print(f"Class prediction is {pred}. Ground-truth: {gt}")
        max_items_to_print -= 1
        if max_items_to_print == 0:
            break

Class prediction is AbdomenCT. Ground-truth: AbdomenCT
Class prediction is BreastMRI. Ground-truth: BreastMRI
Class prediction is ChestCT. Ground-truth: ChestCT
Class prediction is CXR. Ground-truth: CXR
Class prediction is Hand. Ground-truth: Hand
Class prediction is HeadCT. Ground-truth: HeadCT
Class prediction is HeadCT. Ground-truth: HeadCT
Class prediction is CXR. Ground-truth: CXR
Class prediction is ChestCT. Ground-truth: ChestCT
Class prediction is BreastMRI. Ground-truth: BreastMRI
